In [ ]:
from pypdf import PdfReader

reader = PdfReader("Tommy Nguyen Resume 2 Pages.pdf")

print(len(reader.pages))

resulting_text = reader.pages[0].extract_text()

# Removing all "comments" brought over from Word to PDF when converting. Forgot to remove them beforehand. Preprocessing the text here.
resulting_list = resulting_text.split("\n")
for i in range(len(resulting_list)):
    index_of_comments = resulting_list[i].lower().find("comment")
    if index_of_comments >= 0:
        resulting_list[i] = resulting_list[i].replace(resulting_list[i][index_of_comments:-1], "")

    if "graduated august 2023..." in resulting_list[i].lower():
        resulting_list[i] = ""
    if "undergraduate courses." == resulting_list[i].lower().strip():
        resulting_list[i] = ""

processed_text = "\n".join(resulting_list)
print(processed_text)